In [16]:
import pandas as pd
import numpy as np

In [17]:
df_original = pd.read_csv("CSV/proyecto4.csv")


In [18]:
df_original.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country',
       'release_year', 'rating', 'duration', 'listed_in', 'plataforma'],
      dtype='object')

In [19]:
# Crear tabla con los tipos 
data_tipos = [{'type_id': 1, 'type_name': 'Movie'}, {'type_id': 2, 'type_name': 'TV Show'}]

df_tipos = pd.DataFrame(data_tipos)

# Guardar la tabla como un archivo CSV
df_tipos.to_csv('tabla_tipos.csv', index=False)

In [20]:
# Crear tabla de directores

df_directores = df_original[['show_id', 'director']].copy()


df_directores['director'] = df_directores['director'].fillna('') 

df_directores = df_directores.assign(director=df_directores['director'].str.split(', ')).explode('director')

df_directores['director'] = df_directores['director'].str.strip()

df_directores = df_directores[df_directores['director'] != '']

df_directores_final = df_directores['director'].drop_duplicates().reset_index(drop=True).to_frame()

df_directores_final = df_directores_final.reset_index()
df_directores_final = df_directores_final.rename(columns={'index': 'director_id', 'director': 'director_name'})

# Asegurar que el ID empiece en 1
df_directores_final['director_id'] = df_directores_final['director_id'] + 1

# Guardar la tabla única de directores
df_directores_final.to_csv('tabla_directores_referencia.csv', index=False)

In [21]:
# Crear tabla de cast

df_actores = df_original[['show_id', 'cast']].copy()

df_actores['cast'] = df_actores['cast'].fillna('') 

# Se crea una fila única para cada actor, repitiendo el show_id.
df_actores = df_actores.assign(cast=df_actores['cast'].str.split(', ')).explode('cast')

df_actores['cast'] = df_actores['cast'].str.strip()

df_actores = df_actores[df_actores['cast'] != '']

df_actores_final = df_actores['cast'].drop_duplicates().reset_index(drop=True).to_frame()


df_actores_final = df_actores_final.reset_index()
df_actores_final = df_actores_final.rename(columns={'index': 'cast_id', 'cast': 'cast_name'})

# Asegurar que el ID empiece en 1 
df_actores_final['cast_id'] = df_actores_final['cast_id'] + 1

# 4. Guardar la tabla única de actores
df_actores_final.to_csv('tabla_actores_referencia.csv', index=False)

In [22]:
# TABLA PLATAFORMA. 
plataforma = df_original['plataforma']
valores_unicos = df_original['plataforma'].unique() #Sacar los valores unicos 
plataforma_unicos = pd.DataFrame({'plataforma': valores_unicos})
plataforma_unicos['plataforma_id'] = range(1, len(plataforma_unicos) + 1)
plataforma_unicos = plataforma_unicos[['plataforma_id', 'plataforma']]

In [23]:
year = df_original['release_year']
valores_year = df_original['release_year'].unique()
year_tratado = pd.DataFrame({'release_year': valores_year})
year_unicos = year_tratado.sort_values(by='release_year', ascending=False)
year_unicos['release_year_id'] = range(1,len(year_unicos) + 1)
year_unicos = year_unicos[['release_year_id', 'release_year']]

In [24]:
# Tratamiento de country como columna. Lo primero que hacemos es tratar los missing values. Después, normalizamos todo para poder quedarnos con el primer value que aparece
df_original['country'] = df_original['country'].replace({'Missing values': 'Desconocido'})
df_original['country'] = df_original['country'].astype(str).apply(lambda x: x.split(',')[0].strip())
df_original['country'] = df_original['country'].apply(lambda x: x.split(',')[0].strip())

In [25]:
# TABLA COUNTRY
country = df_original['country']
valores_country = df_original['country'].unique()
country_tratado = pd.DataFrame({'country': valores_country})
country_unicos = country_tratado.sort_values(by='country', ascending=True)
country_unicos['country_id'] = range(1,len(country_unicos) + 1)
country_unicos = country_unicos[['country_id', 'country']]

In [26]:
# Tratamiento de genero como columna. Lo primero que hacemos es normalizar para poder tratar los values como valores unicos. Después, nos hemos quedado con los primeros valores 
df_original['listed_in'] = df_original['listed_in'].astype(str).apply(lambda x: x.split(',')[0].strip())
df_original['listed_in'] = df_original['listed_in'].apply(lambda x: x.split(',')[0].strip())

In [27]:
#Tabla Genero
genero = df_original['listed_in']
valores_genero = df_original['listed_in'].unique()
genero_tratado = pd.DataFrame({'listed_in': valores_genero})
genero_unicos = genero_tratado.sort_values(by='listed_in', ascending=True)
genero_unicos['listed_in_id'] = range(1,len(genero_unicos) + 1)
genero_unicos = genero_unicos[['listed_in_id', 'listed_in']]

In [28]:
# TABLA TITULOS
titulo = df_original['title']
titulo_unicos = df_original['title'].unique() #Sacar los valores unicos 
titulo_unicos = pd.DataFrame({'title': titulo_unicos})
titulo_unicos['title_id'] = range(1, len(titulo_unicos) + 1)
titulo_unicos = titulo_unicos[['title_id', 'title']]

In [29]:
plataforma_unicos.to_csv('CSV/plataforma.csv', index=False)
year_unicos.to_csv('CSV/año.csv', index=False)
country_unicos.to_csv('CSV/pais.csv', index=False)
genero_unicos.to_csv('CSV/genero.csv', index=False)
titulo_unicos.to_csv('CSV/titulo.csv', index=False)